# RAG app using GPT

Based on https://youtu.be/BrsocJb-fAo?si=ecQjtT3cPC9xGvip

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Loading the model

In [2]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")

In [4]:
model.invoke("Which team won Champions League in 2014?")

AIMessage(content='Real Madrid won the Champions League in 2014.', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 17, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Which team won Champions League in 2014?")

'Real Madrid won the Champions League in 2014.'

### Prompt templates

In [6]:
from langchain.prompts import ChatPromptTemplate

template = """  Answer the question based on the provided context below.
If you cannot find any information related so you cannot answer it, just
reply 'I do not know'.

Context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Ramos scored in CL final in 2014.",
              question="Who scored in Champions League final 2014?")

"Human:   Answer the question based on the provided context below.\nIf you cannot find any information related so you cannot answer it, just\nreply 'I do not know'.\n\nContext: Ramos scored in CL final in 2014.\nQuestion: Who scored in Champions League final 2014?\n"

In [7]:
chain = prompt | model | parser
chain.invoke({
    "context": "Ramos scored in CL final in 2014.",
    "question": "Who scored in Champions League final 2014?"
})

'Ramos scored in the Champions League final in 2014.'

#### Combining chains

(Question + Context) > Prompt > (Query) Model > (Response) Parser >> Answer

(>>Answer + Language) Translation Prompt > (Query) Model > (Response) Parser >> Answer

In [9]:
translation_prompt = ChatPromptTemplate.from_template(
    template="Translate the {answer} to {language}"
)

In [11]:
from operator import itemgetter

input = {"answer": chain, "language": itemgetter("language")}

translation_chain = input | translation_prompt | model | parser

translation_chain.invoke({
   # For the previous chain that now serves as input
    "context": "Sergio Ramos scored in the last minute. It was a header.",
    "question": "How was the last goal of the CL final 2024?",
    # For the translation chain
    "language": "Spanish"
}
)

'El último gol de la final de la Liga de Campeones de 2024 fue anotado por Sergio Ramos con un cabezazo en el último minuto.'

 ## Testing with own custom context

In [13]:
cv_context_file = "cv.txt"

with open(cv_context_file, "r") as f:
    context = f.read()

In [14]:
chain.invoke({
    "context": context,
    "question": "Which is the highest level of education of Roberto?"
})

'The highest level of education of Roberto is an Advanced Master in Artificial Intelligence from KU Leuven (Belgium).'

In [15]:
chain.invoke({
    "context": context,
    "question": "How many languages does Roberto speak?"
})

'Roberto speaks two languages, Spanish and English.'

In [17]:
chain.invoke({
    "context": context,
    "question": "Has Roberto had any international experience?"
})

'Yes, Roberto has had international experience. He worked as an Artificial Intelligence Researcher intern at VITO in Leuven, Belgium, and also completed an Advanced Master in Artificial Intelligence at KU Leuven in Belgium.'

In [18]:
chain.invoke({
    "context": context,
    "question": "Tell me the 3 most strenghts of Roberto in terms of tools."
})

"1. Roberto's strength in using containerization and orchestration tools like Docker and Kubernetes.\n2. Roberto's proficiency in working with various MLOps tools such as Miflow, Kubeflow, and MinIO.\n3. Roberto's skills in using relational and time-series databases like PostgreSQL and InfluxDB."

In [16]:
chain.invoke({
    "context": context,
    "question": "Does Roberto like football?"
})

'I do not know.'